In [7]:
import pandas as pd
import numpy as np

In [24]:
#upload dataset
train_data = pd.read_csv("D:/EIC-Code/00-Python/Machine-Learning-HW/Perceptron/bank-note/train.csv", header = None, names = ['variance','skewness','curtosis','entropy','y'])
test_data = pd.read_csv("D:/EIC-Code/00-Python/Machine-Learning-HW/Perceptron/bank-note/test.csv", names = ['variance','skewness','curtosis','entropy','y'])

In [ ]:
class StandardPerceptron: #only when you have class, you will need self. A class is a blueprint for creating objects. It defines a set of attributes and methods that the objects created from it will have.
    def __init__(self, max_epoches, learning_rate):
        self.max_epoches = max_epoches
        self.weights = None
        self.r = learning_rate
    
    def fit(self, X, y):
        n = X.shape[1] # number of features
        self.weights = np.zeros(n) #inital weights

        for epoch in range(self.max_epoches):
            for i in range(len(X)):
                y_predict = np.sign(X[i].dot(self.weights)) #Shuffle the data, how to shuffle the data
                if y_predict != y[i]:
                    self.weights  += self.r*y[i]*X[i] #wt+1 ←wt + r (yi xi)

    def predict(self, X): #we need to use self anytime 
        y_predict = np.sign(X.dot(self.weights))
        return y_predict
    
    def scores(self, X, y):
        predictions = self.predict(X) 
        errors = predictions != y
        average_prediction_errors = np.mean(errors) #calculate the mean of true value
        return average_prediction_errors

In [43]:
if __name__ == "__main__":
    features = ['variance','skewness','curtosis','entropy']
    outcome = 'y'

    X_train = train_data[features].values #change to matrix multiple
    y_train = train_data[outcome].values
    X_test = test_data[features].values
    y_test = test_data[outcome].values

    #set hyperparameters
    epoches = 10
    learning_rate = 0.01 #learning rate is a small positive number less than or equal to 1, do I need to set the learning rate here?

    #fit model
    SPmodel = StandardPerceptron(epoches, learning_rate)
    SPmodel.fit(X_train, y_train)

    #return weights and model performance
    weights = SPmodel.weights
    average_score = SPmodel.scores(X_test, y_test)
    print('final weights', weights)
    print('model performance', average_score)

final weights [-0.18831286 -0.14068138 -0.17351032 -0.11696084]
model performance 0.658


In [64]:
class VotePerceptron: #only when you have class, you will need self.
    def __init__(self, max_epoches, learning_rate):
        self.max_epoches = max_epoches
        self.r = learning_rate
        self.weights = []
        self.c = []
    
    def fit(self, X, y):
        n = X.shape[1] # number of features
        w = np.zeros(n) #inital weights
        m = 0 
        c = 0

        for epoch in range(self.max_epoches):
            for i in range(len(X)):
                y_predict = X[i].dot(w)
                test = y[i]*y_predict
                if test <= 0: # for misclassification
                    w += self.r*y[i]*X[i]
                    m += 1 
                    c = 1 #if these is one missclassification, we will reset the count
                else:
                    c += 1 #c is the count of correc predictions
                    self.weights.append(w.copy())
                    self.c.append(c)
        #print(len(self.weights), len(self.c))

    def predict(self, X): #we need to use self anytime
        predictions = []
        for i in range(len(X)):
            final_predict = 0
            for ci, wi in zip(self.c, self.weights):
                one_predict = np.sign(np.dot(wi, X[i]))
                sum_ci_ypredict = ci * one_predict
                final_predict += sum_ci_ypredict
            predictions.append(np.sign(final_predict))
        return predictions

    def scores(self, X, y):
        predictions = self.predict(X)
        errors = predictions != y
        average_prediction_errors = np.mean(errors)
        return average_prediction_errors

In [63]:
if __name__ == "__main__":
    max_epoches = 10
    learning_rate = 0.01

    #define the model
    votemodel = VotePerceptron(max_epoches, learning_rate)
    votemodel.fit(X_train, y_train)

    distinct_weights = votemodel.weights
    distinct_count = votemodel.c

    model_performance = votemodel.scores(X_test, y_test)
    print("the list of the distinct weight and count", [i for i in zip(distinct_weights, distinct_count)])
    print('model performance', model_performance)

the list of the distinct weight and count [(array([-0.051635 , -0.019907 ,  0.0445816, -0.0376682]), 2), (array([-0.051635 , -0.019907 ,  0.0445816, -0.0376682]), 3), (array([-0.051635 , -0.019907 ,  0.0445816, -0.0376682]), 4), (array([-0.051635 , -0.019907 ,  0.0445816, -0.0376682]), 2), (array([-0.051635 , -0.019907 ,  0.0445816, -0.0376682]), 3), (array([-0.051635 , -0.019907 ,  0.0445816, -0.0376682]), 4), (array([-0.051635 , -0.019907 ,  0.0445816, -0.0376682]), 5), (array([-0.051635 , -0.019907 ,  0.0445816, -0.0376682]), 6), (array([-0.051635 , -0.019907 ,  0.0445816, -0.0376682]), 2), (array([-0.051635 , -0.019907 ,  0.0445816, -0.0376682]), 2), (array([-0.051635 , -0.019907 ,  0.0445816, -0.0376682]), 3), (array([-0.051635 , -0.019907 ,  0.0445816, -0.0376682]), 2), (array([-0.051635 , -0.019907 ,  0.0445816, -0.0376682]), 3), (array([-0.051635 , -0.019907 ,  0.0445816, -0.0376682]), 2), (array([-0.051635 , -0.019907 ,  0.0445816, -0.0376682]), 2), (array([-0.051635 , -0.0199

In [71]:
class averageperceptron: #only when you have class, you will need self.
    def __init__(self, max_epoches, learning_rate):
        self.max_epoches = max_epoches
        self.r = learning_rate
        self.weights = None
        self.a = None
    
    def fit(self, X, y):
        n = X.shape[1] # number of features
        self.weights = np.zeros(n) #inital weights
        self.a = np.zeros(n)

        for epoch in range(self.max_epoches):
            for i in range(len(X)):
                y_predict = X[i].dot(self.weights)
                test = y[i]*y_predict
                if test <= 0: # for misclassification
                    self.weights += self.r*y[i]*X[i]
                self.a += self.weights #store the accurate weights

    def predict(self, X): #we need to use self anytime
        predictions = []
        for i in range(len(X)):
            prediction = np.sign(np.dot(X[i], self.a))
            predictions.append(prediction)
        
        return predictions

    def scores(self, X, y):
        predictions = self.predict(X)
        errors = predictions != y
        average_prediction_errors = np.mean(errors)
        return average_prediction_errors

In [72]:
if __name__ == "__main__":
    max_epoches = 10
    learning_rate = 0.01

    #define the model
    votemodel = averageperceptron(max_epoches, learning_rate)
    votemodel.fit(X_train, y_train)

    distinct_weights = votemodel.weights

    model_performance = votemodel.scores(X_test, y_test)
    print("the final weights", distinct_weights)
    print('model performance', model_performance)

the final weights [-0.18831286 -0.14068138 -0.17351032 -0.11696084]
model performance 0.61
